# Import Libraries

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import torch
from PIL import Image
import cv2


# Pastikan menggunakan GPU jika tersedia
if tf.config.list_physical_devices('GPU'):
    print("GPU Tersedia")
else:
    print("GPU Tidak Tersedia, menggunakan CPU")

GPU Tersedia


# Data Preparation

In [3]:
# Path dataset
base_path = '/kaggle/input/human-fall-datasets/train'
test_path = '/kaggle/input/human-fall-datasets/test'

# Fungsi untuk membaca dataset
def process_directory(base_path, label_map):
    data = []
    for subject in os.listdir(base_path):
        subject_path = os.path.join(base_path, subject)
        if os.path.isdir(subject_path):
            for category in os.listdir(subject_path):
                category_path = os.path.join(subject_path, category)
                label = label_map.get(category, None)
                if label is not None:
                    for subcategory in os.listdir(category_path):
                        subcategory_path = os.path.join(category_path, subcategory)
                        for filename in os.listdir(subcategory_path):
                            if filename.endswith('.jpg'):
                                file_path = os.path.join(subcategory_path, filename)
                                data.append({
                                    "subject": subject,
                                    "category": category,
                                    "subcategory": subcategory,
                                    "file_path": file_path,
                                    "label": label
                                })
    return data
#encode kategori class output
label_map = {
    "fall": 1,
    "non_fall": 0
}


# Proses data
train_data = process_directory(base_path, label_map)
df = pd.DataFrame(train_data)
test_data = []
for filename in os.listdir(test_path):
    if filename.endswith('.jpg'):
        file_path = os.path.join(test_path, filename)
        test_data.append({"id": filename, "file_path": file_path})
test_df = pd.DataFrame(test_data)


# load yolov5 model
modelYolo = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Preprocessing

In [6]:
# Split data
X = df['file_path']
y = df['label']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Preprocessing gambar
def yolo_crop(image_path):
    # Decode bytes to string if needed
    if isinstance(image_path, bytes):
        image_path = image_path.decode('utf-8')
    
    # Perform YOLO detection
    results = modelYolo(Image.open(image_path))
    detections = results.xyxy[0]  # [xmin, ymin, xmax, ymax, confidence, class]
    
    for *box, conf, cls in detections:
        if int(cls) == 0:  # Class 0 corresponds to 'person' in the COCO dataset
            xmin, ymin, xmax, ymax = map(int, box)
            # Open image and crop based on detection
            image = Image.open(image_path)
            cropped_image = image.crop((xmin, ymin, xmax, ymax))
            return cropped_image

    # Return the original image if no detection
    return Image.open(image_path)


# Image preprocessing function
def preprocess_image(image_path, img_size=(224, 224)):
    # Use YOLO to crop the image
    cropped_image = yolo_crop(image_path)
    
    # Convert PIL image to a TensorFlow-compatible format
    cropped_image = cropped_image.convert('RGB')  # Ensure 3 channels
    cropped_image = cropped_image.resize(img_size)  # Resize to target dimensions
    img_array = tf.keras.preprocessing.image.img_to_array(cropped_image)
    
    # Preprocess using ResNetV2 preprocessing
    img_array = tf.keras.applications.resnet_v2.preprocess_input(img_array)
    return img_array

# Wrapper to use with TensorFlow dataset
def preprocess_image_wrapper(image_path, label, img_size=(224, 224)):
    img = tf.numpy_function(preprocess_image, [image_path], tf.float32)
    img.set_shape((img_size[0], img_size[1], 3))  # Explicitly set the shape
    return img, label



# Data generator for training
train_gen = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_gen = train_gen.map(lambda x, y: preprocess_image_wrapper(x, y))
train_gen = train_gen.batch(16).prefetch(tf.data.AUTOTUNE)

# Data generator for validation
val_gen = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_gen = val_gen.map(lambda x, y: preprocess_image_wrapper(x, y))
val_gen = val_gen.batch(16).prefetch(tf.data.AUTOTUNE)

print(type(val_gen))

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>


# Arsitektur Model

In [7]:
# Model
def create_model():
    base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Unfreeze beberapa layer terakhir
    for layer in base_model.layers[-30:]:
        layer.trainable = True
    
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    return Model(base_model.input, outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
print("w")

w


In [8]:
#menghilangkan warning untuk model yolo
import warnings
warnings.filterwarnings("ignore")
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6,
        verbose=1
    ),
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy')
]

# Ensemble models
def train_ensemble(n_models=3):
    models = []
    histories = []
    
    for i in range(n_models):
        print(f"\nTraining Model {i+1}/{n_models}")
        model = create_model()
        
        # Buat optimizer baru untuk setiap model
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
        
        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=80,
            callbacks=callbacks
        )
        
        models.append(model)
        histories.append(history)
    
    return models, histories

# Train ensemble
models, histories = train_ensemble()

# Prediksi ensemble
def ensemble_predict(models, data):
    predictions = []
    for model in models:
        pred = model.predict(data)
        predictions.append(pred)
    
    # Rata-rata prediksi
    ensemble_pred = np.mean(predictions, axis=0)
    return ensemble_pred

# Evaluasi model ensemble
val_predictions = ensemble_predict(models, val_gen)
val_pred_classes = (val_predictions > 0.5).astype(int).flatten()

print("\nLaporan Klasifikasi:")
print(classification_report(y_val, val_pred_classes))

# Plot confusion matrix
cm = confusion_matrix(y_val, val_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


Training Model 1/3
234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/80
215/215 ━━━━━━━━━━━━━━━━━━━━ 495s 1s/step - accuracy: 0.8998 - loss: 18.8645 - val_accuracy: 0.9942 - val_loss: 13.1367 - learning_rate: 1.0000e-04
Epoch 2/80
215/215 ━━━━━━━━━━━━━━━━━━━━ 177s 821ms/step - accuracy: 0.9899 - loss: 11.6163 - val_accuracy: 0.9953 - val_loss: 7.5927 - learning_rate: 1.0000e-04
Epoch 3/80
215/215 ━━━━━━━━━━━━━━━━━━━━ 175s 816ms/step - accuracy: 0.9958 - loss: 6.5955 - val_accuracy: 1.0000 - val_loss: 4.0862 - learning_rate: 1.0000e-04
Epoch 4/80
215/215 ━━━━━━━━━━━━━━━━━━━━ 171s 794ms/step - accuracy: 0.9993 - loss: 3.5044 - val_accuracy: 1.0000 - val_loss: 2.0787 - learning_rate: 1.0000e-04
Epoch 5/80
215/215 ━━━━━━━━━━━━━━━━━━━━ 170s 790ms/step - accuracy: 1.0000 - loss: 1.7635 - val_accuracy: 1.0000 - val_loss: 1.0098 - learning_rate: 1.0000e-04
Epoch 6/80
215/215 ━━━━━━━━━━━━━━━━━━━━ 170s 792ms/step - accuracy: 1.0000 - loss: 0.8508 - val_accuracy: 1.0000 - val_loss: 0.

# Evaluasi Model

In [9]:
# Plot training history
def plot_training_history(histories):
    plt.figure(figsize=(12, 4))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    for i, history in enumerate(histories):
        plt.plot(history.history['accuracy'], label=f'Train {i+1}')
        plt.plot(history.history['val_accuracy'], label=f'Val {i+1}')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    for i, history in enumerate(histories):
        plt.plot(history.history['loss'], label=f'Train {i+1}')
        plt.plot(history.history['val_loss'], label=f'Val {i+1}')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

plot_training_history(histories)

# File Submission dan Testing

In [33]:
def preprocess_image(image_path, img_size=(224, 224)):
    # YOLO crop wrapper for TensorFlow
    def yolo_crop_wrapper(image_path):
    # Decode from bytes to string if necessary
        if isinstance(image_path, np.ndarray):
            image_path = image_path.item()  # Get the string from NumPy scalar
        
        # Perform YOLO cropping
        cropped_image = yolo_crop(image_path)
        
        # Resize the cropped image to the target size
        cropped_image = cropped_image.resize((224, 224))  # Ensure (128, 128) dimensions
        
        # Convert to NumPy array with float32 type
        cropped_image = np.array(cropped_image, dtype=np.float32)
        return cropped_image


    # Use tf.numpy_function to call the Python-based function
    img_array = tf.numpy_function(yolo_crop_wrapper, [image_path], tf.float32)
    img_array = tf.reshape(img_array, (*img_size, 3))  # Explicitly set shape

    # Preprocess using ResNetV2 preprocessing
    img_array = tf.keras.applications.resnet_v2.preprocess_input(img_array)
    return img_array

def preprocess_image_wrapper2(image_path, img_size=(224, 224)):
    img = tf.numpy_function(preprocess_image, [image_path], tf.float32)
    img.set_shape((img_size[0], img_size[1], 3))  # Explicitly set the shape
    return img

# Dataset for test images
test_gen = tf.data.Dataset.from_tensor_slices(test_df['file_path'].values)
test_gen = test_gen.map(preprocess_image_wrapper2)
test_gen = test_gen.batch(16).prefetch(tf.data.AUTOTUNE)

# Run predictions
test_predictions = ensemble_predict(models, test_gen)

# Post-process predictions and save results
test_df['label'] = (test_predictions > 0.5).astype(int)
test_df[['id', 'label']].to_csv('/kaggle/working/predictions1.csv', index=False)
print("Predictions saved yes.")


135/135 ━━━━━━━━━━━━━━━━━━━━ 80s 581ms/step
135/135 ━━━━━━━━━━━━━━━━━━━━ 65s 482ms/step
135/135 ━━━━━━━━━━━━━━━━━━━━ 65s 479ms/step
[[    0.99981]
 [    0.73789]
 [   0.029843]
 ...
 [   0.079657]
 [    0.12422]
 [    0.42182]]
Predictions saved yes.


In [19]:
#simpan model
model_save_dir = '/kaggle/working/ensemble_models'
os.makedirs(model_save_dir, exist_ok=True)  # Create directory if it doesn't exist

for i, model in enumerate(models):
    model.save(os.path.join(model_save_dir, f'modelsub_{i}.h5'))